In [1]:
import crystal_toolkit

/home/azka/anaconda3/envs/full_automate/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from ase.io import read

from ase.build import add_adsorbate
from ase.geometry import get_distances
import numpy as np
import pandas as pd
# import random
from pathlib import Path

from pymatgen.core import Structure, Lattice, Molecule
from pymatgen.core.operations import SymmOp

from full_automate import CreateInterface

# Input

In [3]:
nmc_rate_batch = [[1,0,0], [1,1,1], [5,3,2], [5,2,3], [9, 1.5, 0.5], [9, 0.5, 1.5]]

supercell = Structure.from_file("../_input/quantumatk/POSCAR_sc_li_outer")
supercell_mp_id = "25411"       # expand this later
# supercell_ase = read("quantumatk/POSCAR_sc_li_outer")
# supercell_ase.cell

molecule_folder = Path("../_input/fg/")
# molecule = Molecule.from_file("../_input/fg/diurethane.xyz")

# ## alternative
# from pymatgen.io.ase import AseAtomsAdaptor

# molecule_ase = read("fg/triamine.xyz")
# molecule_ase = AseAtomsAdaptor.get_molecule(atoms)

# molecule_centered = molecule.get_centered_molecule()

# molecule_ase_xyz = CreateInterface.align_principal_axes(molecule_ase)
# molecule_ase.get_positions()

/home/azka/anaconda3/envs/full_automate/lib/python3.11/site-packages/pymatgen/io/vasp/inputs.py:474: BadPoscarWarning: Ignoring selective dynamics tag, as no ionic degrees of freedom were fixed.
  warnings.warn(


In [ ]:
molecule_paths = list(molecule_folder.glob("*.xyz"))

metadata = {}

##### Step 1: substitute element and vary NMC rate #####
for nmc_rate in nmc_rate_batch:
    nmc_structure = CreateInterface.substitute_ni_atoms(supercell, nmc_rate, seed=None)

    for molecule_path in molecule_paths:
        molecule_name = molecule_path.stem  # filename without extension
        molecule = Molecule.from_file(molecule_path)
        
        label = print(f"{supercell_mp_id}_{''.join(map(str, nmc_rate))}_{molecule_name}")

        entry = {}

        # entry = {
        #     "molecule_name": molecule_name,
        #     "molecule": molecule.copy()
        # }

        entry["supercell_mp_id"] = supercell_mp_id
        entry["nmc_rate"] = nmc_rate
        entry["molecule_name"] = molecule_name
        
        ##### Step 2: get longest axis of molecule #####
        molecule_axis = CreateInterface.get_principal_axis(molecule)
        # entry["molecule_axis"] = molecule_axis

        ##### Step 3: determine the thickest lattice direction #####
        supercell_frac_coords = np.array([site.frac_coords for site in supercell])
        spread = np.ptp(supercell_frac_coords, axis=0)  # peak-to-peak: max - min along each axis
        thickest_idx = np.argmax(spread)
        lattice_vec_thickest_idx = supercell.lattice.matrix[thickest_idx]

        # Get supercell direction (unit vector)
        supercell_unitvec = lattice_vec_thickest_idx / np.linalg.norm(lattice_vec_thickest_idx)      # returns: array([-2.65311532e-08,  1.00000000e+00,  0.00000000e+00])

        ##### Step 4: Rotate molecule to align molecule_axis with supercell_unitvec #####
        # Assuming molecule_axis and supercell_unitvec are normalized vectors
        rotation_axis = np.cross(molecule_axis, supercell_unitvec)      # array([-0.02777762,  0.99961261,  0.        ])

        # Skip rotation if vectors are already aligned
        if np.linalg.norm(rotation_axis) > 1e-3:
            rotation_axis /= np.linalg.norm(rotation_axis)
            rotation_angle_rad = np.arccos(np.clip(np.dot(molecule_axis, supercell_unitvec), -1.0, 1.0))    # 1.5690559577350829
            rotation_angle_deg = np.rad2deg(rotation_angle_rad)     # 89.90028419807753

            rotation = SymmOp.from_origin_axis_angle(origin=[0, 0, 0],
                                                    axis=rotation_axis,
                                                    angle=rotation_angle_deg,
                                                    angle_in_radians=False)
            molecule_rotated = molecule.copy()
            molecule_rotated.apply_operation(rotation)
        else:
            molecule_rotated = molecule.copy()

        ##### Step 4a: fine-tuning flipping through specific direction #####
        ## Mirror over the y-axis (flip the sign of y-coordinates)
        molecule_cart_coords_flipped = molecule_rotated.cart_coords.copy()
        if molecule_name in ["diurethane", "triamine"]:
            molecule_cart_coords_flipped[:, 0] *= -1  # Flip x-axis
            molecule_cart_coords_flipped[:, 1] *= -1  # Flip y-axis
            # molecule_cart_coords_flipped[:, 2] *= -1  # Flip z-axis
        elif molecule_name in ["triester", "triether", "triol"]:
            pass
        elif molecule_name == "dicarbonate":    # Note: disscuss this again!
            pass

        # Create a new Molecule with flipped coordinates
        molecule_rotated_flipped = Molecule(species=molecule_rotated.species, coords=molecule_cart_coords_flipped)
        # molecule_rotated_flipped

        ##### Step 5: translate molecule so it sits 3 Å above the top of supercell #####
        ## Get longest lattice vector of supercell
        supercell_lengths = supercell.lattice.abc
        longest_idx = np.argmax(supercell_lengths)
        longest_unitvec = np.array([1 if i == longest_idx else 0 for i in range(3)])
        
        supercell_cart_coords = np.array([site.coords for site in supercell])
        molecule_cart_coords = np.array(molecule_rotated_flipped.cart_coords)

        # # --- Get topmost atom in slab along vacuum direction ---
        # supercell_top = np.max(supercell_cart_coords[:, longest_idx])

        # # Center of the supercell in y++
        # supercell_min = np.min(supercell_cart_coords[:, thickest_idx])
        # supercell_max = np.max(supercell_cart_coords[:, thickest_idx])
        # supercell_center = np.mean(supercell_cart_coords, axis=0)       # didnt really like this
        # molecule_center = np.mean(molecule_cart_coords, axis=0)

        # --- Get molecule centroid and shift ---
        molecule_com = molecule_rotated_flipped.center_of_mass
        molecule_at_origin = molecule_rotated_flipped.translate_sites(range(len(molecule_rotated_flipped)), -molecule_com)

        # combined_structure = supercell.copy()
        # for site in molecule_at_origin.sites:
        #     combined_structure.append(site.specie, site.coords, coords_are_cartesian=True)

        # combined_structure = combined_structure.get_sorted_structure()

        # # Save result
        # combined_structure.to(filename="molecule_at_origin_center.cif")

        # supercell center in 2D excluding the longest direction 
        # longest_unitvec_inverse = 1 - longest_unitvec   # inversion of longest_unitvec
        supercell_2d_center = supercell.lattice.matrix.sum(axis=0) / 2  # Question: how do they compute for y direc?
        supercell_2d_center[longest_idx] = 0

        # Place molecule center in 2D center of supercell
        molecule_translated_x_y = molecule_at_origin.copy()
        translation = supercell_2d_center - molecule_com
        molecule_translated_x_y.translate_sites(range(len(molecule_at_origin)), translation)

        # combined_structure = supercell.copy()
        # for site in molecule_translated_x_y.sites:
        #     combined_structure.append(site.specie, site.coords, coords_are_cartesian=True)

        # combined_structure = combined_structure.get_sorted_structure()

        # # Save result
        # combined_structure.to(filename="molecule_at_origin_center_translated_x_y.cif")

        # Get z-coordinate of lowest atom in molecule
        z_low_molecule = min(site.z for site in molecule_translated_x_y.sites)
        # print(z_low_molecule)

        # Then lift molecule along thickest_idx axis to be 3 Å above top

        # Get z-coordinate of lowest atom in molecule
        z_low_molecule = min(site.z for site in molecule_translated_x_y.sites)

        # z_top_supercell = np.max([site.coords[longest_idx] for site in supercell.sites])
        z_top_supercell = np.max(supercell_cart_coords[:, longest_idx])
        # molecule_bottom = np.min([site[longest_idx] for site in molecule_rotated.cart_coords])
        # z_shift = z_top_supercell - molecule_bottom + 3.0  # 3 Å vacuum
        translation_z = np.zeros(3)
        translation_z[longest_idx] = z_top_supercell - z_low_molecule + 3.0  # 3 Å vacuum

        molecule_translated_x_y_z = molecule_translated_x_y.copy()
        molecule_translated_x_y_z.translate_sites(range(len(molecule_translated_x_y)), translation_z)

        combined_structure = supercell.copy()
        for site in molecule_translated_x_y_z.sites:
            combined_structure.append(site.specie, site.coords, coords_are_cartesian=True)

        combined_structure = combined_structure.get_sorted_structure()
        entry["combined_structure"] = combined_structure

        # Save result
        combined_structure.to(filename=f"../_results/{label}.cif")
        combined_structure.to(filename=f"../_results/POSCAR_{label}")

        metadata[label] = entry

/home/azka/anaconda3/envs/full_automate/lib/python3.11/site-packages/pymatgen/core/structure.py:3874: FutureWarning: We strongly discourage using a default `mode`, it would be set to `r` now but would not be allowed after 2025-06-01
  with zopen(filename) as file:
/home/azka/anaconda3/envs/full_automate/lib/python3.11/site-packages/pymatgen/io/cif.py:1724: FutureWarning: We strongly discourage using implicit binary/text `mode`, and this would not be allowed after 2025-06-01. I.e. you should pass t/b in `mode`.
  with zopen(filename, mode=mode) as file:


Original Ni indices: [27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]
Shuffled Ni indices: [47, 48, 49, 34, 27, 29, 45, 43, 42, 36, 52, 28, 30, 35, 46, 39, 44, 31, 50, 53, 37, 51, 32, 40, 38, 41, 33]
NMC ratio: [1. 0. 0.]
n_ni: 27, n_mn: 0, n_co: 0
25411_100_diurethane
25411_100_dicarbonate
25411_100_triamine
25411_100_triester
25411_100_triol
25411_100_triether
Original Ni indices: [27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]
Shuffled Ni indices: [37, 52, 27, 48, 38, 36, 43, 32, 50, 47, 51, 31, 30, 33, 35, 49, 39, 42, 40, 53, 34, 41, 46, 28, 45, 29, 44]
NMC ratio: [0.33333333 0.33333333 0.33333333]
n_ni: 9, n_mn: 9, n_co: 9
25411_111_diurethane
25411_111_dicarbonate
25411_111_triamine
25411_111_triester
25411_111_triol
25411_111_triether
Original Ni indices: [27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 5

In [5]:
# # Then lift molecule along longest axis to be 3 Å above top
# molecule_translated_x_y_z = molecule_translated_x_y.copy()
# offset = np.zeros(3)
# offset[longest_idx] = supercell_max - np.min(molecule_cart_coords[:, longest_idx]) + 3.0
# molecule_translated_x_y_z.translate_sites(range(len(molecule_translated_x_y)), offset)

# print("Note: doesn't work")

In [6]:
metadata

{None: {'supercell_mp_id': '25411',
  'nmc_rate': [9, 0.5, 1.5],
  'molecule_name': 'triether',
  'combined_structure': Structure Summary
  Lattice
      abc : 8.716036849615 17.570024033931006 26.187220967135
   angles : 90.0 90.0 90.0000015201231
   volume : 4010.336601945985
        A : 8.716036849615 0.0 0.0
        B : -4.66153e-07 17.570024033931 0.0
        C : 0.0 0.0 26.187220967135
      pbc : True True True
  PeriodicSite: Li (-3.449e-06, 14.6, 2.062) [-3.513e-07, 0.8307, 0.07876]
  PeriodicSite: Li (-1.841e-06, 8.739, 2.062) [-1.846e-07, 0.4974, 0.07876]
  PeriodicSite: Li (-2.331e-07, 2.883, 2.062) [-1.797e-08, 0.1641, 0.07876]
  PeriodicSite: Li (1.453, 13.15, 4.125) [0.1667, 0.7487, 0.1575]
  PeriodicSite: Li (1.453, 7.298, 4.125) [0.1667, 0.4154, 0.1575]
  PeriodicSite: Li (1.453, 1.441, 4.125) [0.1667, 0.08203, 0.1575]
  PeriodicSite: Li (-3.216e-06, 11.71, 6.187) [-3.333e-07, 0.6667, 0.2363]
  PeriodicSite: Li (-1.608e-06, 5.857, 6.187) [-1.667e-07, 0.3333, 0.2363]
  